# Data Preprocessing Tools

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Importing the dataset

In [2]:
dataset = pd.read_csv('data.csv')
X = dataset.iloc[:, :-1].values # all rows, all columns except the last one
y = dataset.iloc[:, -1].values # all rows, last column

# test
print(X)
print(y)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]
['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

In [3]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values= np.nan, strategy= 'mean')
# missing data
# numerical - average, median (average recommended)
# categorial - the most frequent value
imputer.fit(X[:, 1:3]) # only columns with numerical values - calculates necessary paremeters and store them intenrally
X[:, 1:3] =  imputer.transform(X[:, 1:3]) # returns columns with updated values

# test
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

In [4]:
# one-hot encoding
# take the number of possible values and add that number of columns (0 or 1 value)
# France - 1 0 0
# Spain - 0 1 0
# Germany - 0 0 1

### Encoding the Independent Variable

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough') # transformers = (what we want to do, type of transormation, index of 
# columns to transform), remainder = what to with other columns - in this case, keep them as they are
X = np.array(ct.fit_transform(X)) # returns the updated matrix of features X and convert it to np array
# test
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the Dependent Variable

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y) # converts directly Y to encoded version
# test
print(y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into the Training set and Test set

In [7]:
# NOTE: feature scaling goes after splitting dataset into the training/test set !!!
# test set should be totally new to ML model before its creation, which means if we apply feature scaling before spliiting
# we'll take values/nature of the test set into consideration and we should not do that. They should be independent.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1) # test_size = percent size of the dataset 
# that test set will have - 20% of the dataset is test set, 80% is training set; randomly taken
# random state is seed of the random function, if we make it determinstic, we'll get the same distribution over and over 
# test
print(X_train)
print(y_train)
print(X_test)
print(y_test)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]
[0 1 0 0 1 1 0 1]
[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]
[0 1]


## Feature Scaling

In [8]:
# To avoid some features dominate, and other are not even cosidered. Not necessary all the time.
# Standardization: x_stand = (x-mean(x))/stddev(x) estimated: [-3, 3] -> good tecnique all the time
# Normalization: x_norm = (x-min(x))/(max(x)-min(x)) estimated: [0,1] -> when we have normal distribution in most of the features
# again, we apply techniques to x_training -> calculate mean and stdev, and we use the same values for test dataset

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # no arguments
# NOTE: we do not apply standardization on "dummy" features - got from OneHot encdoing e.g., so already in [-3,3]. Also, we would lose the meaning of those 
# variables in that way
# fit calculates the params - mean and stdev
# transform applies scaling

X_train[:, -3:] = scaler.fit_transform(X_train[:, -3:])
# NOTE: use the same params - params got from previous fit
X_test[:, -3:] = scaler.transform(X_test[:, -3:])
print(X_train)
print(X_test)


[[0.0 0.0 1.2909944487358056 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 -0.7745966692414834 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 -0.7745966692414834 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.2909944487358056 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.2909944487358056 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 -0.7745966692414834 1.1475343068237058 1.232653363453549]
 [0.0 1.0 -0.7745966692414834 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 -0.7745966692414834 -0.7401495441200351 -0.5646194287757332]]
[[0.0 1.0 -0.7745966692414834 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 -0.7745966692414834 -0.44973664397484414 0.2056403393225306]]
